In [1]:
from collections import OrderedDict
import datetime as dt
from pathlib import Path

import pandas as pd
import numpy as np
from highcharts import Highchart



In [ ]:
# Load real data

DATA_DIR = Path('~')/'tumeke_cycle_space'/'finances'
paths = [
    'transactions_20140701--20150630.csv',
    'transactions_20150701--20160630.csv',
    'transactions_20160701--20161130.csv'
    ]
paths = [DATA_DIR/p for p in paths]
parser = lambda date: pd.datetime.strptime(date, '%d-%m-%Y')
frames = [pd.read_csv(p, parse_dates=['Date'], date_parser=parser)
  for p in paths]
f = pd.concat(frames)

# Rename columns for convenience
columns1 = {
  c: c.lower().strip().replace(' ', '_').replace('(', '').replace(')', '')
  for c in f.columns
  }
columns2 = {
  'memo/description': 'memo',
  'amount_credit': 'credit',
  'amount_debit': 'debit',
  }
transactions = f.rename(columns=columns1).rename(columns=columns2)
transactions.tail().T


In [2]:
# Or load mock data

def make_sample_transactions(start_date, end_date):
    r = pd.date_range(start_date, end_date, freq='3D')
    f = pd.DataFrame(r, columns=['date'])
    n = len(r)
    f['credit'] = np.random.randint(0, 100, n)
    f['debit'] = np.random.randint(0, 100, n)
    f['balance'] = (f['credit'] - f['debit']).cumsum()
    return f

transactions = make_sample_transactions('2015-01-01', '2017-01-01')
transactions.tail()

,date,credit,debit,balance
239,2016-12-18,0,46,-251
240,2016-12-21,38,65,-278
241,2016-12-24,52,88,-314
242,2016-12-27,53,52,-313
243,2016-12-30,67,52,-298


In [11]:
def summarize(transactions, freq='MS'):
    """
    Given a data frame of bank transactions with at least the columns
    
    - ``'date'``: datetime
    - ``'credit'``: nonnegative float
    - ``'debit'``: nonnegative float
    - ``'balance'``: float
    
    summarize the transactions at the given frequency (Pandas frequency string),
    summing credits, summing debits, and taking the last balance for each period.
    Return the resulting data frame.
    """
    cols = ['date', 'credit', 'debit', 'balance'] 
    f = transactions[cols].copy()
    if freq is None:
        g = {}
        g['date'] = f['date'].min()
        g['credit'] = f['credit'].sum()
        g['debit'] = f['debit'].sum()
        g['balance'] = f['balance'].iat[-1]
        g = pd.DataFrame(g, index=[0])
    else:
        g = f.set_index('date').resample(freq).agg({
          'credit': 'sum',
          'debit': 'sum',
          'balance': 'last', 
          }).fillna(0).reset_index()
    
    return g[cols].copy()

def plot(summary, currency='NZD', width=700, height=None):
    """
    Given a transaction summary of the form output by :func:`summarize`,
    plot the summary using Python HighCharts.
    Include the given currency units (string; e.g. 'NZD') in the plot labels.
    """
    f = summary.copy()
    chart = Highchart()

    # HighCharts kludge: use categorical x-axis to display dates properly
    dates = f['date'].map(lambda x:x.strftime('%Y-%m-%d')).unique()
    dates = sorted(dates.tolist())

    if currency:
        y_text = 'Money ({!s})'.format(currency)
    else:
        y_text = 'Money'
    
    options = {
        'lang': {
            'thousandsSep': ','
        },
        'chart' : {},
        'title': {
            'text': 'Account Summary'
        },
        'xAxis': {
            'type': 'category',
            'categories': dates,
        },
        'yAxis': {
            'title': {
                'text': y_text,
            }
        },
        'tooltip': {
            'headerFormat': '<b>{point.key}</b> ' +
              '(period start)<table>',
            'pointFormat': '''
              <tr>
              <td style="padding-right:1em">{series.name}</td>
              <td style="text-align:right">{point.y:,.0f} ''' + currency +\
              '''
              </td>
              </tr>
              ''',    
            'useHTML': True,
            'shared': True,
        },
        'plotOptions': {
            'column': {
                'pointPadding': 0,
                'borderWidth': 1,
                'borderColor': '#333333',
            }
        },
        'credits': {
                'enabled': False,
            },
    }

    if width is not None:
        options['chart']['width'] = width

    if height is not None:
        options['chart']['height'] = height

    chart.set_dict_options(options)
    for (col, opts) in [
      ('credit', {'series_type': 'column', 'color': '#8da0cb'}),
      ('debit', {'series_type': 'column', 'color': '#fc8d62'}),
      ('balance', {'series_type': 'line', 'color': '#555'}),
      ]:
        chart.add_data_set(f[col].values.tolist(), name=col, **opts)

    return chart

In [18]:
f = summarize(transactions, freq='Q')
print(f)
plot(f)


        date  credit  debit  balance
0 2015-03-31    1552   1608      -56
1 2015-06-30    1641   1478      107
2 2015-09-30    1560   1616       51
3 2015-12-31    1553   1472      132
4 2016-03-31    1476   1620      -12
5 2016-06-30    1351   1674     -335
6 2016-09-30    1466   1428     -297
7 2016-12-31    1670   1671     -298
